# WL Ising 2D
Clarisse Scofield | 2018054699

In [2]:
%pip install numba

     |████████████████████████████████| 3.4 MB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 25.3 MB 30.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from numba import jit
import numpy as np

Niter=10000000  # Número máximo de passos de Monte Carlo (iterações do algoritmo)
L=6            # Dimensão linear da rede
flatness = 0.8  # Condição para zerar o histograma e diminuir lnf quando
                # min(Histogram) > average(Histogram)*flattness

N=L*L           # Total de sítios da rede

print("Simulação do modelo de Ising 2D pelo método de Wang-Landau")
print("Rede quadrada",L,"x",L)

@jit(nopython=True)
def estado_ini(N):
    #Gera um estadon inicial aleatório para rede
    s = np.zeros(N,dtype=np.int8)
    for i in range(N):
        s[i] = np.sign(2*np.random.random()-1)
    return s

@jit(nopython=True)
def vizinhos(L,N):
    #Define a tabela de vizinhos 
    viz = np.zeros((N,4),dtype=np.int16)
    for k in range(N):
        viz[k,0]=k+1
        if (k+1) % L == 0: viz[k,0] = k+1-L
        viz[k,1] = k+L 
        if k > (N-L-1): viz[k,1] = k+L-N
        viz[k,2] = k-1 
        if k % L == 0: viz[k,2] = k+L-1
        viz[k,3] = k-L 
        if k < L: viz[k,3] = k+N-L
    return viz

@jit(nopython=True)
def energia(s,viz):
    #Calcula a energia da configuração s
    ener = 0 
    for i in range(N):
        h = s[viz[i,0]]+s[viz[i,1]]
        ener -= s[i]*h
    ener = int((ener+2*N)/4)
    return ener

@jit(nopython=True)
def minh(H):
    #Calcula o menor valor de H excluindo as energias proibidas
    minh=H[0]
    for i in range(2,N-1):
        if H[i] < minh: minh=H[i]
    if H[-1] < minh: minh=H[-1]
    return minh

@jit(nopython=True)
def wang_landau(N,Niter,flatness,ener,s):
    # Algoritmo de Wang-Landau
    lnge = np.zeros(N+1,dtype=np.float64)
    H = np.zeros(N+1,dtype=np.int16)
    Hc = np.zeros(N+1,dtype=np.int16)
    mmicro = np.zeros(N+1,dtype=np.float64)
    lnf = 1.0
    m = s.sum()
    for it in range(Niter):
        #Iterações do algoritmo
        for imc in range(N):
            #Passo de Monte Carlo - percorre toda a rede. trocar n spins de direcao
            k = np.random.randint(0,N-1)
            h = s[viz[k,0]]+s[viz[k,1]]+s[viz[k,2]]+s[viz[k,3]] # soma dos vizinhos
            ener2 = ener + int(s[k]*h*0.5)
            #print(lnge[ener]-lnge[ener2])
            if lnge[ener]>lnge[ener2]:
                s[k] = -s[k] 
                ener = ener2
                m -= 2*s[k]
            else:
                p = np.exp(lnge[ener]-lnge[ener2])
                if np.random.random() < p: 
                    s[k] = -s[k] 
                    ener = ener2
                    m -= 2*s[k]
            H[ener] += 1
            lnge[ener] += lnf
            mmicro[ener] += abs(m)
        if it%1000 == 0:
            hmed = float(H.sum())/float((N-1))
            hmin = minh(H)
            if hmin > (flatness*hmed):
                Hc += H
                H = np.zeros(N+1,dtype=np.int16)
                lnf = 0.5*lnf
                print("Histograma flat!",lnf)
            if it%1000000 == 0: print("Iteração número",it)
        if lnf < 0.00000001: break
        
    mmicro = mmicro/Hc
    lnge = lnge - lnge[0]+np.log(2)
    return lnge,mmicro,ener,s

In [ ]:
#Execucao
Niter=10000000  # Número máximo de passos de Monte Carlo (iterações do algoritmo)
L=6            # Dimensão linear da rede
flatness = 0.8  # Condição para zerar o histograma e diminuir lnf quando
                # min(Histogram) > average(Histogram)*flattness
N = L * L

s = estado_ini(N)
viz = vizinhos(L, N)
ener = energia(s, viz)
wang = wang_landau(N,Niter,flatness,ener,s)
print(wang)

In [15]:
import pandas as pd

#ln g(E) para L = 24
df = pd.read_csv('lnge.dat',
            header=None, sep='\s\s+', engine='python')
print(df)

       0          1           2
0      0   0.693147  568.166727
1      2   7.035616  564.341234
2      3   7.730097  562.516755
3      4  12.708448  562.129834
4      5  14.091362  559.830706
..   ...        ...         ...
570  571  14.121778   83.993426
571  572  12.741752   84.015692
572  573   7.766827   84.000000
573  574   7.057542   84.025871
574  576   0.757825   84.000000

[575 rows x 3 columns]
